## Clustering in Python

### Objective:
In this vignette, we will be implementing the K-means, Spectral, and Agglomerative clustering algorithms to better understand the nuances of each and better analyze the behaviors of users in our dataset.

### Data:

##### Importing modules:

##### Partitioning data:

## Clustering Methods:
We will be comparing three different clustering methods today: k-means clustering, spectral clustering, and agglomerative clustering.

#### K-Nearest-Neighbors (knn):

#### Spectral Clustering:
Spectral clustering is a powerful and versatile algorithm used for partitioning data into groups based on similarity. Unlike traditional clustering methods like k-means, which rely on geometric properties in Euclidean space, spectral clustering leverages the eigenvalues and eigenvectors of a similarity matrix derived from the data. This approach enables it to identify clusters with complex, non-convex shapes that might not be easily separable using conventional methods. By constructing a graph representation of the data where nodes represent data points and edges reflect their pairwise similarity, spectral clustering transforms the problem into one of graph partitioning. The algorithm computes the Laplacian matrix of the graph and uses its eigenvectors to embed the data in a lower-dimensional space where the clustering structure becomes more apparent.

This makes spectral clustering particularly effective in applications involving image segmentation, social network analysis, and other domains with intricate relational data. Its reliance on pairwise similarities and graph theory provides a robust framework for uncovering hidden patterns in complex datasets.

We will apply spectral clustering on our dataset below!

#### Agglomerative Clustering: 